In [5]:
from dotenv import load_dotenv
import os
import openai 
from PIL import Image
import requests

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar se a chave foi carregada
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada no arquivo .env")

# Configurar o cliente OpenAI com a chave carregada
openai.api_key = api_key

In [6]:
client = openai.Client()

### Gerar imagens

In [7]:
nome="robo_praia"
model="dall-e-3"
prompt="Crie uma imagem com dois robôs em uma praia lendo livro de finanças"
size="1024x1024"
quality="hd"
style="vivid"

In [8]:
resposta = client.images.generate(
    model=model,
    prompt=prompt,
    size=size,
    quality=quality,
    style=style,
    n=1
)

In [9]:
resposta.data[0].revised_prompt

"Generate an image of two robots sitting on a beach. The robots are engaged in reading a finance book. They are sitting under a colorful beach umbrella and there's a calm turquoise sea in the background. Decorate the scene with seashells and starfish around them. The sand has a golden hue and the sky exhibits shades of orange and pink, indicative of the setting sun. On the distant horizon, we see silhouettes of ships, marking a picturesque scene."

In [10]:
nome_arquivo = f"files{nome}_{model}_{quality}_{style}.jpg"
image_url = resposta.data[0].url 
image_data = requests.get(image_url).content 
with open(nome_arquivo, "wb") as f: 
    f.write(image_data)

### Image to Text

In [11]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role":"user",
        "content":[
            {"type":"text","text":"Descreva a imagem fornecida"},
            {"type": "image_url","image_url":{
                "url":"https://harus.ind.br/wp-content/uploads/2023/02/5-praias-brasileiras.jpg"
            }}
        ]
    }]
)

In [12]:
resposta.choices[0].message.content

'A imagem mostra uma bela praia ao entardecer. No primeiro plano, há areia clara e algumas palmeiras que se inclinam em direção ao mar. O céu exibe tons quentes de laranja, rosa e roxo, refletindo no nível da água, que está calmo e sereno. Ao fundo, pode-se ver uma colina e algumas construções, provavelmente restaurantes ou acomodações. A atmosfera é tranquila e convida ao relaxamento.'

### Extraia placa de carro 

In [16]:
texto = "Qual é a placa do carro? Devolva apenas a placa do carro!"

In [18]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role":"user",
        "content":[
            {"type":"text","text":texto},
            {"type": "image_url","image_url":{
                "url":"https://quatrorodas.abril.com.br/wp-content/uploads/2018/09/imagem-e1536873662306.jpg?quality=70&strip=info&w=720&crop=1"
            }}
        ]
    }]
)

In [19]:
resposta.choices[0].message.content

'R102A19'

### OCR

In [20]:
texto = "O que está escrito na imagem?"

In [21]:
resposta = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": texto},
            {"type": "image_url", "image_url": {
                "url": "https://www.rbsdirect.com.br/imagesrc/17752154.jpg?w=700"
            }}
        ]
    }],
    max_tokens=1000
)

In [22]:
resposta.choices[0].message.content

'O que está escrito na imagem é:\n\n"ISSO É UM PEDIDO:\n\nQue você seja muito feliz no seu grande dia.\nEste vestido tem mãos de uma brasileira, que gera emprego e movimenta a economia local.\n\nFoi um prazer fazê-lo para você.\n\nElianna  \nEstilista  \nBelo Horizonte, 1/5"'

In [23]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What are in these images? Is there any difference between them?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

In [24]:
response.choices[0].message.content

"The two images appear to be identical, both depicting a scenic view of a wooden pathway leading through lush green grass under a bright blue sky with clouds. Without being able to analyze the images further, I can't determine if there are any subtle differences or variations between them."